# Day 02: Database Design for Tick Data and Time Series

## Week 22: System Design for Quantitative Finance

---

### Learning Objectives
- Understand database architecture choices for financial time series data
- Design efficient schemas for tick data (trades, quotes, order book)
- Implement indexing strategies for time-based queries
- Master partitioning and data retention policies
- Optimize queries for backtesting and analytics workloads

### Why Database Design Matters in Quant Finance
- **Data Volume**: A single liquid stock can generate 100K+ ticks per day
- **Query Patterns**: Backtests require fast range queries over years of data
- **Real-time Ingestion**: Production systems must handle thousands of updates/second
- **Cost Efficiency**: Proper design can reduce storage costs by 10x or more

---

## Table of Contents
1. [Import Required Libraries](#1-import-required-libraries)
2. [Database Options for Financial Data](#2-database-options)
3. [Tick Data Schema Design](#3-tick-data-schema)
4. [OHLCV Time Series Schema](#4-ohlcv-schema)
5. [Database Connection with Connection Pooling](#5-database-connection)
6. [SQLAlchemy ORM Models](#6-orm-models)
7. [Indexing Strategy for Time Series](#7-indexing-strategy)
8. [Data Partitioning by Date](#8-partitioning)
9. [High-Throughput Data Insertion](#9-data-insertion)
10. [Query Optimization Patterns](#10-query-optimization)
11. [Data Retention Policies](#11-retention-policies)
12. [Best Practices Summary](#12-best-practices)

## 1. Import Required Libraries <a id="1-import-required-libraries"></a>

In [1]:
# Core Python libraries
import os
import sys
from datetime import datetime, timedelta, date
from decimal import Decimal
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field
from enum import Enum
import json
import time
import hashlib

# Data manipulation
import numpy as np
import pandas as pd

# SQLAlchemy for ORM and database operations
from sqlalchemy import (
    create_engine, MetaData, Table, Column, Index,
    Integer, BigInteger, String, Float, DateTime, Date,
    Numeric, Boolean, Text, JSON, Enum as SQLEnum,
    ForeignKey, UniqueConstraint, CheckConstraint,
    PrimaryKeyConstraint, func, text, event, inspect
)
from sqlalchemy.orm import (
    declarative_base, sessionmaker, Session, relationship,
    scoped_session
)
from sqlalchemy.dialects.postgresql import (
    TIMESTAMP, INTERVAL, ARRAY, JSONB, UUID, BYTEA,
    insert as pg_insert
)
from sqlalchemy.pool import QueuePool
from sqlalchemy.exc import IntegrityError, OperationalError

# For connection pooling and async operations
from contextlib import contextmanager
import threading
from queue import Queue

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: f'{x:,.6f}')

print("✅ All libraries imported successfully!")
print(f"📅 Notebook Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

ModuleNotFoundError: No module named 'sqlalchemy'

## 2. Database Options for Financial Data <a id="2-database-options"></a>

### Overview of Database Technologies for Time Series

| Database | Type | Best For | Throughput | Compression | Notes |
|----------|------|----------|------------|-------------|-------|
| **PostgreSQL** | Relational | General purpose, joins | Medium | Low | Most versatile |
| **TimescaleDB** | Time-series (PG extension) | Time-series with SQL | High | High | Best balance |
| **InfluxDB** | Time-series native | Metrics, IoT | Very High | Very High | No joins |
| **QuestDB** | Time-series native | Ultra-low latency | Extreme | High | SQL-like |
| **kdb+/q** | Column-store | HFT, derivatives | Extreme | Custom | Industry standard, expensive |
| **Arctic (MongoDB)** | Document + versioned | Research, versioned data | Medium | High | Python-native |
| **ClickHouse** | Column-store OLAP | Analytics, aggregations | Very High | Very High | Great for reads |
| **DuckDB** | Embedded OLAP | Local analytics | High | Medium | In-process |

### Key Selection Criteria
1. **Query patterns**: OLTP (many small queries) vs OLAP (few large queries)
2. **Write patterns**: Batch vs streaming ingestion
3. **Data volume**: GB vs TB vs PB scale
4. **Latency requirements**: Milliseconds vs seconds acceptable
5. **SQL compatibility**: Full SQL vs limited query language
6. **Operational complexity**: Managed vs self-hosted

In [ ]:
# Database Comparison Helper Class
@dataclass
class DatabaseOption:
    """Compare database options for financial data storage."""
    name: str
    db_type: str
    write_throughput: str  # rows/second
    query_latency: str
    compression_ratio: str
    sql_support: str
    best_use_case: str
    cost: str
    
    def __str__(self):
        return f"{self.name}: {self.best_use_case}"

# Define database options for quant finance
DATABASE_OPTIONS = [
    DatabaseOption(
        name="PostgreSQL + TimescaleDB",
        db_type="Time-series (SQL)",
        write_throughput="100K-500K rows/sec",
        query_latency="10-100ms",
        compression_ratio="10-20x",
        sql_support="Full SQL",
        best_use_case="General quant research, backtesting",
        cost="Free / Enterprise"
    ),
    DatabaseOption(
        name="QuestDB",
        db_type="Time-series native",
        write_throughput="1M+ rows/sec",
        query_latency="<10ms",
        compression_ratio="10-15x",
        sql_support="SQL-like (partial)",
        best_use_case="Real-time analytics, HFT monitoring",
        cost="Free / Enterprise"
    ),
    DatabaseOption(
        name="InfluxDB",
        db_type="Time-series native",
        write_throughput="500K rows/sec",
        query_latency="10-50ms",
        compression_ratio="20-50x",
        sql_support="InfluxQL / Flux",
        best_use_case="Metrics, monitoring, simple time-series",
        cost="Free / Cloud pricing"
    ),
    DatabaseOption(
        name="kdb+/q",
        db_type="Column-store",
        write_throughput="5M+ rows/sec",
        query_latency="<1ms",
        compression_ratio="5-10x",
        sql_support="q language",
        best_use_case="HFT, derivatives, real-time trading",
        cost="$$$$ (very expensive)"
    ),
    DatabaseOption(
        name="Arctic (MongoDB)",
        db_type="Document store",
        write_throughput="50K-100K rows/sec",
        query_latency="50-200ms",
        compression_ratio="5-15x",
        sql_support="Python API",
        best_use_case="Research, versioned datasets",
        cost="Free"
    ),
    DatabaseOption(
        name="ClickHouse",
        db_type="Column-store OLAP",
        write_throughput="1M+ rows/sec",
        query_latency="50-500ms",
        compression_ratio="15-30x",
        sql_support="SQL",
        best_use_case="Large-scale analytics, batch processing",
        cost="Free / Cloud pricing"
    ),
]

# Display comparison
print("=" * 80)
print("DATABASE OPTIONS FOR QUANTITATIVE FINANCE")
print("=" * 80)
for db in DATABASE_OPTIONS:
    print(f"\n📊 {db.name}")
    print(f"   Type: {db.db_type}")
    print(f"   Write: {db.write_throughput}")
    print(f"   Latency: {db.query_latency}")
    print(f"   Compression: {db.compression_ratio}")
    print(f"   Best for: {db.best_use_case}")
    print(f"   Cost: {db.cost}")

## 3. Tick Data Schema Design <a id="3-tick-data-schema"></a>

### What is Tick Data?
Tick data represents the most granular level of market data - every individual trade and quote update.

### Types of Tick Data
1. **Trade Ticks**: Actual executed transactions (price, volume, timestamp)
2. **Quote Ticks**: Best bid/ask updates (bid price, ask price, sizes)
3. **Order Book Snapshots**: Full depth of book at specific intervals
4. **Order Book Deltas**: Changes to the order book (adds, modifies, deletes)

### Key Schema Considerations
- **Timestamp precision**: Nanosecond for HFT, microsecond for most use cases
- **Price representation**: Decimal vs float (Decimal for accuracy)
- **Symbol normalization**: Consistent ticker format across exchanges
- **Exchange identification**: Multiple venues for same security
- **Sequence numbers**: Order preservation and gap detection

In [ ]:
# Define the base for ORM models
Base = declarative_base()

# Enums for tick data
class TickType(str, Enum):
    """Type of tick event."""
    TRADE = "trade"
    QUOTE = "quote"
    ORDER_BOOK = "order_book"

class TradeSide(str, Enum):
    """Side of a trade (aggressor side)."""
    BUY = "buy"   # Buyer was aggressor (hit the ask)
    SELL = "sell" # Seller was aggressor (hit the bid)
    UNKNOWN = "unknown"

class Exchange(str, Enum):
    """Trading venue."""
    NYSE = "NYSE"
    NASDAQ = "NASDAQ"
    ARCA = "ARCA"
    BATS = "BATS"
    IEX = "IEX"
    CME = "CME"
    CBOE = "CBOE"
    CRYPTO = "CRYPTO"


# ============================================================================
# TRADE TICK SCHEMA
# ============================================================================
class TradeTick(Base):
    """
    Schema for trade tick data - individual executed transactions.
    
    Storage estimate: ~100 bytes per row
    For 1000 liquid stocks @ 50K trades/day each = 5GB/day uncompressed
    """
    __tablename__ = 'trade_ticks'
    
    # Primary identification (composite primary key for efficiency)
    id = Column(BigInteger, primary_key=True, autoincrement=True)
    
    # Timestamp with nanosecond precision (stored as bigint nanoseconds from epoch)
    # Using BigInteger for nanosecond precision - PostgreSQL TIMESTAMP only supports microseconds
    timestamp_ns = Column(BigInteger, nullable=False, index=True)
    
    # Alternative: Timestamp with microsecond precision (native PostgreSQL)
    timestamp = Column(TIMESTAMP(timezone=True, precision=6), nullable=False)
    
    # Symbol identification
    symbol = Column(String(20), nullable=False, index=True)
    
    # Price as NUMERIC for exact decimal representation (avoiding float precision issues)
    # Precision 18, Scale 8 supports most asset classes (crypto needs more decimals)
    price = Column(Numeric(18, 8), nullable=False)
    
    # Volume/Size
    volume = Column(BigInteger, nullable=False)  # BigInt for large volumes
    
    # Trade metadata
    trade_id = Column(String(50), nullable=True)  # Exchange trade ID if available
    exchange = Column(String(20), nullable=False, default='UNKNOWN')
    
    # Aggressor side (who initiated the trade)
    side = Column(String(10), nullable=True)
    
    # Conditions/flags (e.g., odd lot, out of sequence)
    conditions = Column(ARRAY(String(10)), nullable=True)
    
    # Sequence number for ordering within same timestamp
    sequence_num = Column(BigInteger, nullable=True)
    
    # Value in quote currency (price * volume) - denormalized for query efficiency
    notional_value = Column(Numeric(24, 8), nullable=True)
    
    # Data ingestion metadata
    received_at = Column(TIMESTAMP(timezone=True), default=func.now())
    
    # Composite index for most common query pattern
    __table_args__ = (
        Index('ix_trade_symbol_timestamp', 'symbol', 'timestamp'),
        Index('ix_trade_timestamp_symbol', 'timestamp', 'symbol'),
        Index('ix_trade_exchange_symbol_time', 'exchange', 'symbol', 'timestamp'),
        # Unique constraint to prevent duplicates
        UniqueConstraint('symbol', 'timestamp_ns', 'exchange', 'trade_id', 
                        name='uq_trade_tick'),
        {'schema': 'market_data'}  # Use schema for organization
    )
    
    def __repr__(self):
        return (f"<TradeTick(symbol={self.symbol}, time={self.timestamp}, "
                f"price={self.price}, volume={self.volume})>")


# Display schema details
print("=" * 80)
print("TRADE TICK SCHEMA")
print("=" * 80)
print("\nTable: trade_ticks")
print("\nColumns:")
for column in TradeTick.__table__.columns:
    nullable = "NULL" if column.nullable else "NOT NULL"
    print(f"  {column.name:20} {str(column.type):25} {nullable}")
print("\nIndexes:")
for idx in TradeTick.__table__.indexes:
    cols = ", ".join([c.name for c in idx.columns])
    print(f"  {idx.name}: ({cols})")

In [ ]:
# ============================================================================
# QUOTE TICK SCHEMA
# ============================================================================
class QuoteTick(Base):
    """
    Schema for quote tick data - best bid/ask updates (NBBO or exchange-level).
    
    Storage estimate: ~120 bytes per row
    Quotes update more frequently than trades - can be 10x volume
    """
    __tablename__ = 'quote_ticks'
    
    id = Column(BigInteger, primary_key=True, autoincrement=True)
    
    # Timestamps
    timestamp_ns = Column(BigInteger, nullable=False)
    timestamp = Column(TIMESTAMP(timezone=True, precision=6), nullable=False)
    
    # Symbol
    symbol = Column(String(20), nullable=False)
    
    # Bid side
    bid_price = Column(Numeric(18, 8), nullable=True)
    bid_size = Column(BigInteger, nullable=True)
    bid_exchange = Column(String(20), nullable=True)
    
    # Ask side
    ask_price = Column(Numeric(18, 8), nullable=True)
    ask_size = Column(BigInteger, nullable=True)
    ask_exchange = Column(String(20), nullable=True)
    
    # Derived fields (denormalized for query efficiency)
    spread = Column(Numeric(18, 8), nullable=True)  # ask - bid
    spread_bps = Column(Numeric(10, 4), nullable=True)  # spread in basis points
    mid_price = Column(Numeric(18, 8), nullable=True)  # (bid + ask) / 2
    
    # Quote type (NBBO, exchange-specific, etc.)
    quote_type = Column(String(20), nullable=True)
    
    # Sequence and conditions
    sequence_num = Column(BigInteger, nullable=True)
    conditions = Column(ARRAY(String(10)), nullable=True)
    
    # Metadata
    received_at = Column(TIMESTAMP(timezone=True), default=func.now())
    
    __table_args__ = (
        Index('ix_quote_symbol_timestamp', 'symbol', 'timestamp'),
        Index('ix_quote_timestamp', 'timestamp'),
        UniqueConstraint('symbol', 'timestamp_ns', 'quote_type', name='uq_quote_tick'),
        {'schema': 'market_data'}
    )
    
    def __repr__(self):
        return (f"<QuoteTick(symbol={self.symbol}, bid={self.bid_price}@{self.bid_size}, "
                f"ask={self.ask_price}@{self.ask_size})>")


# ============================================================================
# ORDER BOOK SNAPSHOT SCHEMA
# ============================================================================
class OrderBookSnapshot(Base):
    """
    Schema for full order book snapshots at a point in time.
    
    Two approaches:
    1. Store as JSON blob (flexible, slower queries)
    2. Store as separate rows per level (normalized, faster aggregations)
    
    This uses the JSON approach for flexibility.
    """
    __tablename__ = 'order_book_snapshots'
    
    id = Column(BigInteger, primary_key=True, autoincrement=True)
    
    timestamp_ns = Column(BigInteger, nullable=False)
    timestamp = Column(TIMESTAMP(timezone=True, precision=6), nullable=False)
    
    symbol = Column(String(20), nullable=False)
    exchange = Column(String(20), nullable=False)
    
    # Number of levels captured
    depth = Column(Integer, nullable=False, default=10)
    
    # Order book data as JSONB for flexible querying
    # Structure: {"bids": [[price, size], ...], "asks": [[price, size], ...]}
    book_data = Column(JSONB, nullable=False)
    
    # Summary statistics (denormalized for common queries)
    best_bid = Column(Numeric(18, 8), nullable=True)
    best_ask = Column(Numeric(18, 8), nullable=True)
    total_bid_volume = Column(BigInteger, nullable=True)
    total_ask_volume = Column(BigInteger, nullable=True)
    
    # Imbalance = (bid_vol - ask_vol) / (bid_vol + ask_vol)
    book_imbalance = Column(Numeric(6, 4), nullable=True)
    
    received_at = Column(TIMESTAMP(timezone=True), default=func.now())
    
    __table_args__ = (
        Index('ix_ob_symbol_timestamp', 'symbol', 'timestamp'),
        Index('ix_ob_exchange_symbol_time', 'exchange', 'symbol', 'timestamp'),
        # GIN index for JSONB queries
        Index('ix_ob_book_data', 'book_data', postgresql_using='gin'),
        {'schema': 'market_data'}
    )


print("\n" + "=" * 80)
print("QUOTE TICK SCHEMA")
print("=" * 80)
print("\nTable: quote_ticks")
print("\nKey columns for quotes:")
print("  - bid_price/bid_size: Best bid")
print("  - ask_price/ask_size: Best ask")
print("  - spread_bps: Spread in basis points (pre-calculated)")
print("  - mid_price: (bid + ask) / 2")

print("\n" + "=" * 80)
print("ORDER BOOK SNAPSHOT SCHEMA")
print("=" * 80)
print("\nTable: order_book_snapshots")
print("\nStores full depth of book as JSONB:")
print("  book_data: {")
print('    "bids": [[100.50, 1000], [100.49, 2000], ...],')
print('    "asks": [[100.51, 500], [100.52, 1500], ...]')
print("  }")

## 4. OHLCV Time Series Schema <a id="4-ohlcv-schema"></a>

### OHLCV (Candlestick) Data
Aggregated price data over fixed time intervals:
- **O**pen: First price in the interval
- **H**igh: Highest price in the interval
- **L**ow: Lowest price in the interval
- **C**lose: Last price in the interval
- **V**olume: Total volume traded in the interval

### Common Timeframes
- **Tick-based**: 1 tick, 10 ticks, 100 ticks
- **Time-based**: 1s, 1m, 5m, 15m, 30m, 1h, 4h, 1d, 1w, 1M
- **Volume-based**: 1K volume, 10K volume (equal volume bars)
- **Dollar-based**: $1M notional (equal dollar bars)

In [ ]:
# ============================================================================
# OHLCV CANDLE SCHEMA
# ============================================================================
class Timeframe(str, Enum):
    """Supported timeframes for OHLCV data."""
    SECOND_1 = "1s"
    MINUTE_1 = "1m"
    MINUTE_5 = "5m"
    MINUTE_15 = "15m"
    MINUTE_30 = "30m"
    HOUR_1 = "1h"
    HOUR_4 = "4h"
    DAY_1 = "1d"
    WEEK_1 = "1w"
    MONTH_1 = "1M"

TIMEFRAME_SECONDS = {
    "1s": 1,
    "1m": 60,
    "5m": 300,
    "15m": 900,
    "30m": 1800,
    "1h": 3600,
    "4h": 14400,
    "1d": 86400,
    "1w": 604800,
    "1M": 2592000,  # Approximate
}


class OHLCVCandle(Base):
    """
    Schema for OHLCV candlestick data.
    
    Design choices:
    1. Single table with timeframe column vs separate tables per timeframe
       - Single table: Simpler, but larger indexes
       - Separate tables: More complex, but faster queries per timeframe
    
    2. This design uses a single table with partitioning by timeframe
    
    Storage estimate: ~80 bytes per row
    For 5000 symbols @ 1m bars = 5000 * 1440 * 80 = 576MB/day for 1m data
    """
    __tablename__ = 'ohlcv_candles'
    
    id = Column(BigInteger, primary_key=True, autoincrement=True)
    
    # Candle period start time (ALWAYS use the START of the period)
    timestamp = Column(TIMESTAMP(timezone=True), nullable=False)
    
    # Candle period end time (optional, can be derived from timeframe)
    timestamp_end = Column(TIMESTAMP(timezone=True), nullable=True)
    
    # Symbol
    symbol = Column(String(20), nullable=False)
    
    # Timeframe (1m, 5m, 1h, 1d, etc.)
    timeframe = Column(String(10), nullable=False)
    
    # OHLCV core fields
    open = Column(Numeric(18, 8), nullable=False)
    high = Column(Numeric(18, 8), nullable=False)
    low = Column(Numeric(18, 8), nullable=False)
    close = Column(Numeric(18, 8), nullable=False)
    volume = Column(BigInteger, nullable=False)
    
    # Extended fields for richer analysis
    vwap = Column(Numeric(18, 8), nullable=True)  # Volume-weighted average price
    trade_count = Column(Integer, nullable=True)  # Number of trades
    notional_volume = Column(Numeric(24, 8), nullable=True)  # Dollar volume
    
    # Quote data aggregates (if available)
    avg_spread_bps = Column(Numeric(10, 4), nullable=True)
    avg_bid_size = Column(BigInteger, nullable=True)
    avg_ask_size = Column(BigInteger, nullable=True)
    
    # Return calculation (denormalized for efficiency)
    # return = (close - prev_close) / prev_close
    return_pct = Column(Numeric(12, 8), nullable=True)
    log_return = Column(Numeric(12, 8), nullable=True)
    
    # Volatility measures
    intrabar_range = Column(Numeric(12, 8), nullable=True)  # (high - low) / open
    close_location = Column(Numeric(6, 4), nullable=True)  # (close - low) / (high - low)
    
    # Data quality flags
    is_complete = Column(Boolean, default=True)  # False if partial candle
    data_source = Column(String(50), nullable=True)
    
    # Timestamps
    created_at = Column(TIMESTAMP(timezone=True), default=func.now())
    updated_at = Column(TIMESTAMP(timezone=True), onupdate=func.now())
    
    __table_args__ = (
        # Primary lookup pattern: symbol + timeframe + timestamp
        Index('ix_ohlcv_symbol_tf_time', 'symbol', 'timeframe', 'timestamp'),
        # For time-range queries across all symbols
        Index('ix_ohlcv_tf_time_symbol', 'timeframe', 'timestamp', 'symbol'),
        # Unique constraint
        UniqueConstraint('symbol', 'timeframe', 'timestamp', name='uq_ohlcv_candle'),
        # Check constraints for data quality
        CheckConstraint('high >= low', name='check_high_gte_low'),
        CheckConstraint('high >= open AND high >= close', name='check_high_is_max'),
        CheckConstraint('low <= open AND low <= close', name='check_low_is_min'),
        CheckConstraint('volume >= 0', name='check_volume_positive'),
        {'schema': 'market_data'}
    )
    
    def __repr__(self):
        return (f"<OHLCVCandle(symbol={self.symbol}, tf={self.timeframe}, "
                f"time={self.timestamp}, O={self.open}, H={self.high}, "
                f"L={self.low}, C={self.close}, V={self.volume})>")


# ============================================================================
# DAILY AGGREGATES TABLE (Optimized for daily queries)
# ============================================================================
class DailyBar(Base):
    """
    Separate table for daily data with additional fields.
    Daily data is queried so frequently it deserves its own optimized table.
    """
    __tablename__ = 'daily_bars'
    
    id = Column(BigInteger, primary_key=True, autoincrement=True)
    
    # Date (not timestamp - daily data is date-based)
    date = Column(Date, nullable=False)
    symbol = Column(String(20), nullable=False)
    
    # OHLCV
    open = Column(Numeric(18, 8), nullable=False)
    high = Column(Numeric(18, 8), nullable=False)
    low = Column(Numeric(18, 8), nullable=False)
    close = Column(Numeric(18, 8), nullable=False)
    volume = Column(BigInteger, nullable=False)
    
    # Adjusted prices (for splits and dividends)
    adj_open = Column(Numeric(18, 8), nullable=True)
    adj_high = Column(Numeric(18, 8), nullable=True)
    adj_low = Column(Numeric(18, 8), nullable=True)
    adj_close = Column(Numeric(18, 8), nullable=True)
    adj_volume = Column(BigInteger, nullable=True)
    
    # Adjustment factor (multiply raw by factor to get adjusted)
    adjustment_factor = Column(Numeric(18, 10), default=1.0)
    
    # Additional daily metrics
    vwap = Column(Numeric(18, 8), nullable=True)
    trade_count = Column(Integer, nullable=True)
    notional_volume = Column(Numeric(24, 8), nullable=True)
    
    # Returns
    return_pct = Column(Numeric(12, 8), nullable=True)
    log_return = Column(Numeric(12, 8), nullable=True)
    
    # Corporate actions
    dividend = Column(Numeric(12, 8), nullable=True)
    split_ratio = Column(Numeric(12, 8), nullable=True)
    
    __table_args__ = (
        Index('ix_daily_symbol_date', 'symbol', 'date'),
        Index('ix_daily_date_symbol', 'date', 'symbol'),
        UniqueConstraint('symbol', 'date', name='uq_daily_bar'),
        {'schema': 'market_data'}
    )


print("=" * 80)
print("OHLCV CANDLE SCHEMA")
print("=" * 80)
print("\nTable: ohlcv_candles")
print("\nSupported timeframes:", list(TIMEFRAME_SECONDS.keys()))
print("\nCore fields: open, high, low, close, volume")
print("\nExtended fields:")
print("  - vwap: Volume-weighted average price")
print("  - trade_count: Number of trades in period")
print("  - return_pct: Period return (denormalized)")
print("  - avg_spread_bps: Average bid-ask spread")

print("\n" + "=" * 80)
print("DAILY BAR SCHEMA (Optimized)")
print("=" * 80)
print("\nTable: daily_bars")
print("\nAdditional fields for daily data:")
print("  - Adjusted prices (adj_open, adj_close, etc.)")
print("  - Adjustment factor for splits/dividends")
print("  - Corporate action fields (dividend, split_ratio)")

## 5. Database Connection with Connection Pooling <a id="5-database-connection"></a>

### Connection Pooling Best Practices
- **Pool Size**: Start with 5-10 connections per worker
- **Max Overflow**: Allow 20-50 additional connections for bursts
- **Connection Timeout**: 30 seconds typical
- **Recycle**: Recycle connections every 1-2 hours to avoid stale connections
- **Pre-ping**: Enable connection validation before use

In [ ]:
# ============================================================================
# DATABASE CONNECTION MANAGER
# ============================================================================

@dataclass
class DatabaseConfig:
    """Database connection configuration."""
    host: str = "localhost"
    port: int = 5432
    database: str = "market_data"
    user: str = "trader"
    password: str = "password"
    
    # Connection pool settings
    pool_size: int = 10
    max_overflow: int = 20
    pool_timeout: int = 30
    pool_recycle: int = 3600  # 1 hour
    pool_pre_ping: bool = True
    
    # SSL settings
    ssl_mode: str = "prefer"
    
    def get_connection_string(self, include_password: bool = True) -> str:
        """Generate SQLAlchemy connection string."""
        pwd = self.password if include_password else "****"
        return (
            f"postgresql+psycopg2://{self.user}:{pwd}@"
            f"{self.host}:{self.port}/{self.database}"
            f"?sslmode={self.ssl_mode}"
        )


class DatabaseConnectionManager:
    """
    Manages database connections with connection pooling.
    
    Usage:
        db_manager = DatabaseConnectionManager(config)
        with db_manager.get_session() as session:
            session.query(TradeTick).filter(...)
    """
    
    def __init__(self, config: DatabaseConfig):
        self.config = config
        self._engine = None
        self._session_factory = None
        self._scoped_session = None
        
    def _create_engine(self):
        """Create SQLAlchemy engine with connection pooling."""
        return create_engine(
            self.config.get_connection_string(),
            poolclass=QueuePool,
            pool_size=self.config.pool_size,
            max_overflow=self.config.max_overflow,
            pool_timeout=self.config.pool_timeout,
            pool_recycle=self.config.pool_recycle,
            pool_pre_ping=self.config.pool_pre_ping,
            # Echo SQL for debugging (disable in production)
            echo=False,
            # Future-proof settings
            future=True,
        )
    
    @property
    def engine(self):
        """Get or create the database engine."""
        if self._engine is None:
            self._engine = self._create_engine()
        return self._engine
    
    @property
    def session_factory(self):
        """Get or create session factory."""
        if self._session_factory is None:
            self._session_factory = sessionmaker(
                bind=self.engine,
                autocommit=False,
                autoflush=False,
                expire_on_commit=False
            )
        return self._session_factory
    
    @contextmanager
    def get_session(self) -> Session:
        """
        Get a database session with automatic cleanup.
        
        Usage:
            with db_manager.get_session() as session:
                results = session.query(TradeTick).all()
        """
        session = self.session_factory()
        try:
            yield session
            session.commit()
        except Exception as e:
            session.rollback()
            raise e
        finally:
            session.close()
    
    def get_scoped_session(self):
        """Get thread-local scoped session for multi-threaded apps."""
        if self._scoped_session is None:
            self._scoped_session = scoped_session(self.session_factory)
        return self._scoped_session
    
    def execute_raw(self, sql: str, params: dict = None) -> List[Dict]:
        """Execute raw SQL and return results as list of dicts."""
        with self.engine.connect() as conn:
            result = conn.execute(text(sql), params or {})
            if result.returns_rows:
                return [dict(row._mapping) for row in result]
            return []
    
    def health_check(self) -> bool:
        """Check if database connection is healthy."""
        try:
            self.execute_raw("SELECT 1")
            return True
        except Exception as e:
            print(f"Database health check failed: {e}")
            return False
    
    def get_pool_status(self) -> Dict[str, int]:
        """Get connection pool statistics."""
        pool = self.engine.pool
        return {
            "pool_size": pool.size(),
            "checked_in": pool.checkedin(),
            "checked_out": pool.checkedout(),
            "overflow": pool.overflow(),
            "invalid": pool.invalidatedcount() if hasattr(pool, 'invalidatedcount') else 0
        }
    
    def close(self):
        """Close all connections and dispose of the engine."""
        if self._scoped_session:
            self._scoped_session.remove()
        if self._engine:
            self._engine.dispose()
            self._engine = None


# Example configuration
example_config = DatabaseConfig(
    host="localhost",
    port=5432,
    database="quant_trading",
    user="trader",
    password="secure_password",
    pool_size=10,
    max_overflow=20
)

print("=" * 80)
print("DATABASE CONNECTION MANAGER")
print("=" * 80)
print(f"\nConnection string (sanitized):")
print(f"  {example_config.get_connection_string(include_password=False)}")
print(f"\nPool settings:")
print(f"  Pool size: {example_config.pool_size}")
print(f"  Max overflow: {example_config.max_overflow}")
print(f"  Pool timeout: {example_config.pool_timeout}s")
print(f"  Pool recycle: {example_config.pool_recycle}s")
print(f"  Pre-ping: {example_config.pool_pre_ping}")

## 6. SQLAlchemy ORM Models - Complete Implementation <a id="6-orm-models"></a>

### ORM vs Raw SQL Trade-offs

| Aspect | ORM | Raw SQL |
|--------|-----|---------|
| Development speed | Fast | Slower |
| Type safety | High | Low |
| Query flexibility | Medium | High |
| Performance | ~10-20% overhead | Optimal |
| Maintenance | Easier | Harder |

**Recommendation**: Use ORM for CRUD operations, raw SQL for complex analytics queries.

In [ ]:
# ============================================================================
# COMPLETE TABLE CREATION SCRIPT
# ============================================================================

def create_market_data_schema(engine, drop_existing: bool = False):
    """
    Create all market data tables.
    
    Args:
        engine: SQLAlchemy engine
        drop_existing: If True, drops existing tables first (USE WITH CAUTION!)
    """
    # Create schema if it doesn't exist
    with engine.connect() as conn:
        conn.execute(text("CREATE SCHEMA IF NOT EXISTS market_data"))
        conn.commit()
    
    if drop_existing:
        Base.metadata.drop_all(engine)
        print("⚠️ Dropped existing tables")
    
    # Create all tables
    Base.metadata.create_all(engine)
    print("✅ Created all tables")


def get_table_ddl(table_class) -> str:
    """Get the DDL (CREATE TABLE statement) for a table."""
    from sqlalchemy.schema import CreateTable
    from sqlalchemy.dialects import postgresql
    
    create_stmt = CreateTable(table_class.__table__)
    return str(create_stmt.compile(dialect=postgresql.dialect()))


# Display DDL for each table
print("=" * 80)
print("TABLE DDL STATEMENTS")
print("=" * 80)

tables_to_show = [
    ("TradeTick", TradeTick),
    ("QuoteTick", QuoteTick),
    ("OrderBookSnapshot", OrderBookSnapshot),
    ("OHLCVCandle", OHLCVCandle),
    ("DailyBar", DailyBar),
]

for name, table_class in tables_to_show:
    print(f"\n{'='*40}")
    print(f"-- {name}")
    print("="*40)
    ddl = get_table_ddl(table_class)
    # Truncate for display
    print(ddl[:1500] + "..." if len(ddl) > 1500 else ddl)

## 7. Indexing Strategy for Time Series Queries <a id="7-indexing-strategy"></a>

### Index Types for Time Series

1. **B-tree Index** (default): Best for range queries, equality, ordering
   - Use for: `timestamp`, `symbol`, `exchange`
   
2. **Hash Index**: Fast equality lookups only
   - Use for: exact symbol lookups (rarely better than B-tree)
   
3. **BRIN Index** (Block Range Index): For naturally ordered data
   - Use for: append-only time series (10-100x smaller than B-tree)
   
4. **GIN Index**: For array/JSONB columns
   - Use for: `conditions` array, `book_data` JSONB

### Index Design Principles
- **Leading column**: Most selective or most filtered column first
- **Include columns**: Add frequently selected columns to avoid table lookups
- **Partial indexes**: Index only relevant rows (e.g., last 30 days)
- **Covering indexes**: Include all columns needed by query

In [ ]:
# ============================================================================
# INDEX CREATION STRATEGIES
# ============================================================================

def get_index_creation_sql() -> Dict[str, List[str]]:
    """
    Generate SQL for creating optimized indexes.
    
    Returns dict of table_name -> list of index creation statements
    """
    indexes = {}
    
    # Trade ticks indexes
    indexes['trade_ticks'] = [
        # Primary access pattern: get trades for a symbol in a time range
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_trade_symbol_time_covering
        ON market_data.trade_ticks (symbol, timestamp)
        INCLUDE (price, volume, side)
        """,
        
        # For time-ordered scans across all symbols
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_trade_time_symbol
        ON market_data.trade_ticks (timestamp, symbol)
        """,
        
        # BRIN index for append-only data (very small, fast scans)
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_trade_time_brin
        ON market_data.trade_ticks USING BRIN (timestamp)
        WITH (pages_per_range = 128)
        """,
        
        # Partial index for recent data (last 30 days)
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_trade_recent
        ON market_data.trade_ticks (symbol, timestamp)
        WHERE timestamp > CURRENT_TIMESTAMP - INTERVAL '30 days'
        """,
        
        # Index for exchange-specific queries
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_trade_exchange_symbol_time
        ON market_data.trade_ticks (exchange, symbol, timestamp)
        WHERE exchange IS NOT NULL
        """,
    ]
    
    # OHLCV candles indexes
    indexes['ohlcv_candles'] = [
        # Primary access: symbol + timeframe + time range
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_ohlcv_primary
        ON market_data.ohlcv_candles (symbol, timeframe, timestamp DESC)
        INCLUDE (open, high, low, close, volume)
        """,
        
        # For cross-sectional queries (all symbols at a time)
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_ohlcv_time_tf_symbol
        ON market_data.ohlcv_candles (timestamp, timeframe, symbol)
        """,
        
        # Partial index for 1-minute data (most common)
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_ohlcv_1m_only
        ON market_data.ohlcv_candles (symbol, timestamp DESC)
        INCLUDE (open, high, low, close, volume)
        WHERE timeframe = '1m'
        """,
        
        # BRIN for time-ordered scans
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_ohlcv_time_brin
        ON market_data.ohlcv_candles USING BRIN (timestamp)
        WITH (pages_per_range = 64)
        """,
    ]
    
    # Daily bars indexes
    indexes['daily_bars'] = [
        # Primary access pattern
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_daily_symbol_date
        ON market_data.daily_bars (symbol, date DESC)
        INCLUDE (open, high, low, close, volume, adj_close)
        """,
        
        # Cross-sectional (all symbols on a date)
        """
        CREATE INDEX CONCURRENTLY IF NOT EXISTS ix_daily_date_symbol
        ON market_data.daily_bars (date DESC, symbol)
        """,
    ]
    
    return indexes


def estimate_index_size(
    table_rows: int,
    key_columns_bytes: int,
    include_columns_bytes: int = 0,
    fill_factor: float = 0.7
) -> float:
    """
    Estimate B-tree index size in MB.
    
    Args:
        table_rows: Number of rows in table
        key_columns_bytes: Size of key columns in bytes
        include_columns_bytes: Size of INCLUDE columns in bytes
        fill_factor: Fill factor (default 70%)
    
    Returns:
        Estimated size in MB
    """
    # Each leaf node: 8 bytes pointer + key + include data
    leaf_entry_size = 8 + key_columns_bytes + include_columns_bytes
    
    # Entries per page (8KB page)
    entries_per_page = int((8192 * fill_factor) / leaf_entry_size)
    
    # Number of leaf pages
    leaf_pages = table_rows / entries_per_page
    
    # Internal pages (approximately 1% of leaf pages)
    internal_pages = leaf_pages * 0.01
    
    # Total size in MB
    total_pages = leaf_pages + internal_pages
    size_mb = (total_pages * 8192) / (1024 * 1024)
    
    return size_mb


# Display index strategy
print("=" * 80)
print("INDEX CREATION STRATEGY")
print("=" * 80)

indexes = get_index_creation_sql()
for table, index_sqls in indexes.items():
    print(f"\n📊 Table: {table}")
    print("-" * 40)
    for i, sql in enumerate(index_sqls, 1):
        # Extract index name from SQL
        name = sql.split("IF NOT EXISTS ")[1].split("\n")[0].strip()
        print(f"  {i}. {name}")

# Estimate index sizes for a realistic workload
print("\n" + "=" * 80)
print("INDEX SIZE ESTIMATES (1 year of data)")
print("=" * 80)

estimates = [
    ("trade_ticks", 500_000_000, 28, 24),  # 500M rows, symbol(8)+timestamp(8)+exchange(12), price+vol+side
    ("ohlcv_candles (1m)", 2_000_000_000, 32, 40),  # 2B rows for 1m data
    ("daily_bars", 5_000_000, 14, 48),  # 5M rows (5000 symbols * 1000 days)
]

for table, rows, key_bytes, include_bytes in estimates:
    size = estimate_index_size(rows, key_bytes, include_bytes)
    print(f"\n{table}:")
    print(f"  Rows: {rows:,}")
    print(f"  Estimated covering index size: {size:,.0f} MB ({size/1024:.1f} GB)")

## 8. Data Partitioning by Date <a id="8-partitioning"></a>

### Why Partition Time Series Data?

1. **Query Performance**: Partition pruning eliminates scanning irrelevant data
2. **Data Management**: Easy to drop old partitions (instant vs slow DELETE)
3. **Maintenance**: VACUUM/ANALYZE can run on individual partitions
4. **Storage Tiering**: Move old partitions to cheaper storage

### Partitioning Strategies

| Strategy | Best For | Pros | Cons |
|----------|----------|------|------|
| **Daily** | Tick data, high-volume | Fine-grained, easy cleanup | Many partitions |
| **Weekly** | Medium volume | Good balance | Week boundaries |
| **Monthly** | OHLCV, moderate volume | Fewer partitions | Coarser cleanup |
| **Yearly** | Daily bars, low volume | Minimal overhead | Large partitions |

### TimescaleDB Hypertables
TimescaleDB automates partitioning with "chunks" (partitions) and provides:
- Automatic time-based partitioning
- Efficient chunk pruning
- Native compression (10-20x)
- Continuous aggregates

In [ ]:
# ============================================================================
# POSTGRESQL NATIVE PARTITIONING
# ============================================================================

def generate_partition_ddl(
    table_name: str,
    schema: str,
    partition_column: str,
    partition_type: str = "RANGE",
    start_date: date = None,
    end_date: date = None,
    interval: str = "monthly"  # daily, weekly, monthly
) -> List[str]:
    """
    Generate DDL for creating partitioned table and partitions.
    
    Args:
        table_name: Name of the table
        schema: Schema name
        partition_column: Column to partition by (usually timestamp)
        partition_type: RANGE, LIST, or HASH
        start_date: Start date for partitions
        end_date: End date for partitions
        interval: Partition interval (daily, weekly, monthly)
    
    Returns:
        List of SQL statements
    """
    if start_date is None:
        start_date = date(2020, 1, 1)
    if end_date is None:
        end_date = date(2026, 12, 31)
    
    statements = []
    
    # Example for trade_ticks partitioned table
    parent_ddl = f"""
-- Create partitioned parent table
CREATE TABLE IF NOT EXISTS {schema}.{table_name} (
    id BIGSERIAL,
    timestamp_ns BIGINT NOT NULL,
    timestamp TIMESTAMPTZ NOT NULL,
    symbol VARCHAR(20) NOT NULL,
    price NUMERIC(18, 8) NOT NULL,
    volume BIGINT NOT NULL,
    trade_id VARCHAR(50),
    exchange VARCHAR(20) NOT NULL DEFAULT 'UNKNOWN',
    side VARCHAR(10),
    conditions TEXT[],
    sequence_num BIGINT,
    notional_value NUMERIC(24, 8),
    received_at TIMESTAMPTZ DEFAULT NOW(),
    PRIMARY KEY (id, timestamp)
) PARTITION BY {partition_type} ({partition_column});
    """
    statements.append(parent_ddl)
    
    # Generate partition statements based on interval
    current = start_date
    
    if interval == "daily":
        delta = timedelta(days=1)
        date_format = "%Y%m%d"
    elif interval == "weekly":
        delta = timedelta(weeks=1)
        date_format = "%Y_w%W"
    else:  # monthly
        delta = None  # Handle month increments specially
        date_format = "%Y%m"
    
    while current < end_date:
        if interval == "monthly":
            # Calculate next month
            if current.month == 12:
                next_date = date(current.year + 1, 1, 1)
            else:
                next_date = date(current.year, current.month + 1, 1)
            partition_suffix = current.strftime(date_format)
        else:
            next_date = current + delta
            partition_suffix = current.strftime(date_format)
        
        partition_name = f"{table_name}_{partition_suffix}"
        
        partition_ddl = f"""
CREATE TABLE IF NOT EXISTS {schema}.{partition_name}
PARTITION OF {schema}.{table_name}
FOR VALUES FROM ('{current.isoformat()}') TO ('{next_date.isoformat()}');
        """
        statements.append(partition_ddl)
        
        current = next_date
    
    # Create default partition for out-of-range data
    default_ddl = f"""
CREATE TABLE IF NOT EXISTS {schema}.{table_name}_default
PARTITION OF {schema}.{table_name}
DEFAULT;
    """
    statements.append(default_ddl)
    
    return statements


# ============================================================================
# TIMESCALEDB HYPERTABLE SETUP
# ============================================================================

def get_timescaledb_setup_sql(table_name: str, schema: str = "market_data") -> List[str]:
    """
    Generate SQL for setting up TimescaleDB hypertables.
    
    TimescaleDB provides:
    - Automatic chunk management
    - Native compression
    - Continuous aggregates
    - Data retention policies
    """
    return [
        # Enable TimescaleDB extension
        "CREATE EXTENSION IF NOT EXISTS timescaledb CASCADE;",
        
        # Convert regular table to hypertable
        # chunk_time_interval: how much time data per chunk (partition)
        f"""
        SELECT create_hypertable(
            '{schema}.{table_name}',
            'timestamp',
            chunk_time_interval => INTERVAL '1 day',
            if_not_exists => TRUE,
            migrate_data => TRUE
        );
        """,
        
        # Add space partitioning by symbol for better parallelism
        f"""
        SELECT add_dimension(
            '{schema}.{table_name}',
            'symbol',
            number_partitions => 16,
            if_not_exists => TRUE
        );
        """,
        
        # Enable compression (TimescaleDB 2.0+)
        f"""
        ALTER TABLE {schema}.{table_name} SET (
            timescaledb.compress,
            timescaledb.compress_segmentby = 'symbol',
            timescaledb.compress_orderby = 'timestamp DESC'
        );
        """,
        
        # Create compression policy (compress chunks older than 7 days)
        f"""
        SELECT add_compression_policy(
            '{schema}.{table_name}',
            INTERVAL '7 days',
            if_not_exists => TRUE
        );
        """,
        
        # Create data retention policy (drop chunks older than 2 years)
        f"""
        SELECT add_retention_policy(
            '{schema}.{table_name}',
            INTERVAL '2 years',
            if_not_exists => TRUE
        );
        """,
    ]


# Display partitioning examples
print("=" * 80)
print("POSTGRESQL NATIVE PARTITIONING")
print("=" * 80)

sample_ddl = generate_partition_ddl(
    "trade_ticks_partitioned",
    "market_data",
    "timestamp",
    start_date=date(2024, 1, 1),
    end_date=date(2024, 4, 1),
    interval="monthly"
)

print("\nSample DDL for monthly partitioned trade_ticks:")
for stmt in sample_ddl[:3]:  # Show first 3 statements
    print(stmt)
print(f"\n... and {len(sample_ddl) - 3} more partition statements")

print("\n" + "=" * 80)
print("TIMESCALEDB HYPERTABLE SETUP")
print("=" * 80)

ts_sql = get_timescaledb_setup_sql("trade_ticks")
for i, stmt in enumerate(ts_sql, 1):
    print(f"\n-- Step {i}")
    print(stmt.strip())

## 9. High-Throughput Data Insertion <a id="9-data-insertion"></a>

### Batch Insert Strategies

| Method | Throughput | Use Case |
|--------|------------|----------|
| Individual INSERTs | ~1K rows/sec | Debugging only |
| executemany() | ~10K rows/sec | Small batches |
| execute_values() (psycopg2) | ~100K rows/sec | Production |
| COPY command | ~500K rows/sec | Bulk loads |
| Multi-value INSERT | ~50K rows/sec | Medium batches |

### Key Techniques
1. **Batch size**: 1000-10000 rows per batch
2. **Disable indexes during load**: Add indexes after bulk insert
3. **Unlogged tables**: For temporary staging data
4. **Async writes**: Don't wait for WAL sync
5. **Upsert (ON CONFLICT)**: Handle duplicates efficiently

In [ ]:
# ============================================================================
# HIGH-THROUGHPUT DATA INSERTION
# ============================================================================

class TickDataIngestor:
    """
    High-performance tick data ingestion with batching and upserts.
    """
    
    def __init__(
        self,
        db_manager: DatabaseConnectionManager,
        batch_size: int = 5000,
        flush_interval_seconds: float = 1.0
    ):
        self.db_manager = db_manager
        self.batch_size = batch_size
        self.flush_interval = flush_interval_seconds
        
        self._buffer = []
        self._last_flush = time.time()
        self._lock = threading.Lock()
        
        # Statistics
        self.total_inserted = 0
        self.total_batches = 0
        self.total_errors = 0
    
    def add_tick(self, tick: Dict[str, Any]):
        """Add a tick to the buffer. Flushes when batch is full."""
        with self._lock:
            self._buffer.append(tick)
            
            # Flush if batch size reached or time interval exceeded
            if (len(self._buffer) >= self.batch_size or 
                time.time() - self._last_flush > self.flush_interval):
                self._flush_buffer()
    
    def add_ticks(self, ticks: List[Dict[str, Any]]):
        """Add multiple ticks at once."""
        with self._lock:
            self._buffer.extend(ticks)
            
            while len(self._buffer) >= self.batch_size:
                self._flush_buffer()
    
    def _flush_buffer(self):
        """Flush the buffer to the database."""
        if not self._buffer:
            return
        
        # Take batch_size items from buffer
        batch = self._buffer[:self.batch_size]
        self._buffer = self._buffer[self.batch_size:]
        
        try:
            self._insert_batch(batch)
            self.total_inserted += len(batch)
            self.total_batches += 1
        except Exception as e:
            self.total_errors += 1
            print(f"Error inserting batch: {e}")
            # Could implement retry logic or dead letter queue here
        
        self._last_flush = time.time()
    
    def _insert_batch(self, batch: List[Dict[str, Any]]):
        """
        Insert a batch of ticks using efficient upsert.
        
        Uses PostgreSQL ON CONFLICT DO UPDATE for deduplication.
        """
        if not batch:
            return
        
        # SQL for upsert (INSERT ... ON CONFLICT)
        upsert_sql = """
        INSERT INTO market_data.trade_ticks (
            timestamp_ns, timestamp, symbol, price, volume,
            trade_id, exchange, side, conditions, sequence_num, notional_value
        ) VALUES (
            :timestamp_ns, :timestamp, :symbol, :price, :volume,
            :trade_id, :exchange, :side, :conditions, :sequence_num, :notional_value
        )
        ON CONFLICT (symbol, timestamp_ns, exchange, trade_id) 
        DO UPDATE SET
            price = EXCLUDED.price,
            volume = EXCLUDED.volume,
            side = EXCLUDED.side,
            notional_value = EXCLUDED.notional_value
        """
        
        # Using SQLAlchemy's execute with parameter binding
        # In production, use psycopg2.extras.execute_values for better performance
        with self.db_manager.get_session() as session:
            session.execute(text(upsert_sql), batch)
    
    def flush(self):
        """Force flush any remaining items in buffer."""
        with self._lock:
            while self._buffer:
                self._flush_buffer()
    
    def get_stats(self) -> Dict[str, Any]:
        """Get ingestion statistics."""
        return {
            "total_inserted": self.total_inserted,
            "total_batches": self.total_batches,
            "total_errors": self.total_errors,
            "buffer_size": len(self._buffer),
            "avg_batch_size": self.total_inserted / max(1, self.total_batches),
        }


# ============================================================================
# BULK INSERT METHODS COMPARISON
# ============================================================================

def generate_sample_ticks(n: int, symbol: str = "AAPL") -> List[Dict]:
    """Generate sample tick data for testing."""
    base_time = datetime(2024, 1, 15, 9, 30, 0)
    base_price = 150.0
    
    ticks = []
    for i in range(n):
        timestamp = base_time + timedelta(microseconds=i * 100)
        price = base_price + np.random.randn() * 0.01
        volume = np.random.randint(100, 10000)
        
        ticks.append({
            "timestamp_ns": int(timestamp.timestamp() * 1e9),
            "timestamp": timestamp,
            "symbol": symbol,
            "price": round(price, 4),
            "volume": volume,
            "trade_id": f"T{i:010d}",
            "exchange": "NASDAQ",
            "side": np.random.choice(["buy", "sell"]),
            "conditions": None,
            "sequence_num": i,
            "notional_value": round(price * volume, 2),
        })
    
    return ticks


def get_bulk_insert_sql_examples() -> Dict[str, str]:
    """Return different SQL patterns for bulk inserts."""
    
    return {
        "multi_value_insert": """
-- Multi-value INSERT (moderate performance)
INSERT INTO market_data.trade_ticks (timestamp, symbol, price, volume)
VALUES 
    ('2024-01-15 09:30:00', 'AAPL', 150.25, 100),
    ('2024-01-15 09:30:01', 'AAPL', 150.26, 200),
    ('2024-01-15 09:30:02', 'AAPL', 150.24, 150);
        """,
        
        "copy_command": """
-- COPY command (fastest for bulk loads)
COPY market_data.trade_ticks (timestamp, symbol, price, volume)
FROM '/path/to/data.csv'
WITH (FORMAT csv, HEADER true, DELIMITER ',');
        """,
        
        "copy_from_stdin": """
-- COPY from STDIN (for programmatic access)
-- Python: cursor.copy_expert("COPY table FROM STDIN CSV", file_obj)
COPY market_data.trade_ticks (timestamp, symbol, price, volume)
FROM STDIN WITH (FORMAT csv);
        """,
        
        "upsert_on_conflict": """
-- Upsert with ON CONFLICT (for deduplication)
INSERT INTO market_data.trade_ticks (timestamp, symbol, price, volume, trade_id)
VALUES ('2024-01-15 09:30:00', 'AAPL', 150.25, 100, 'T001')
ON CONFLICT (symbol, timestamp, trade_id) 
DO UPDATE SET 
    price = EXCLUDED.price,
    volume = EXCLUDED.volume;
        """,
        
        "staging_table": """
-- Staging table pattern (for complex ETL)
-- Step 1: Load into unlogged staging table (fast)
CREATE UNLOGGED TABLE staging_ticks (LIKE market_data.trade_ticks);
COPY staging_ticks FROM '/path/to/data.csv' CSV;

-- Step 2: Insert with deduplication
INSERT INTO market_data.trade_ticks
SELECT DISTINCT ON (symbol, timestamp, trade_id) *
FROM staging_ticks
ON CONFLICT DO NOTHING;

-- Step 3: Cleanup
DROP TABLE staging_ticks;
        """,
    }


# Display insertion methods
print("=" * 80)
print("BULK INSERT METHODS")
print("=" * 80)

examples = get_bulk_insert_sql_examples()
for name, sql in examples.items():
    print(f"\n{'='*40}")
    print(f"Method: {name}")
    print("="*40)
    print(sql)

# Sample tick generation demo
print("\n" + "=" * 80)
print("SAMPLE TICK DATA STRUCTURE")
print("=" * 80)

sample_ticks = generate_sample_ticks(3)
for tick in sample_ticks:
    print(f"\n{tick}")

## 10. Query Optimization Patterns <a id="10-query-optimization"></a>

### Common Query Patterns in Quant Finance

1. **Time Range Query**: Get all ticks for a symbol between two timestamps
2. **Cross-Sectional Query**: Get latest price for all symbols at a point in time
3. **Resampling**: Aggregate tick data to OHLCV bars
4. **Rolling Window**: Calculate rolling statistics (SMA, volatility)
5. **ASOF Join**: Match trades to quotes at the time of trade
6. **Gap Detection**: Find missing data periods

### Query Optimization Techniques
- Use EXPLAIN ANALYZE to understand query plans
- Leverage covering indexes (INCLUDE clause)
- Use window functions instead of self-joins
- Prefer CTEs for readability, but inline for performance
- Use materialized views for expensive aggregations